In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import plotly.express as px
import time

In [2]:
from sklearn.model_selection import train_test_split, cross_val_predict, TimeSeriesSplit
from sklearn.linear_model import Ridge
from sklearn.feature_selection import SelectKBest, f_regression
import lightgbm as lgb

In [10]:
from distributed import Client, LocalCluster
import dask.dataframe as dd
from dask_ml.model_selection import train_test_split as dask_train_test_split

In [5]:
from src.metrics import cosine_similarity

# 環境設定

In [6]:
DATA_PATH = "./data"
RESULT_PATH = "./results"

# 関数定義

# 分析

## データ読み込み

In [7]:
train_data = dd.read_parquet(Path(DATA_PATH, "train.parquet"))
test_data = dd.read_parquet(Path(DATA_PATH, "test.parquet"))

## モデルの学習

In [12]:
# データ分割
# TODO: 時系列データであることを考慮すべき？
feature_cols = train_data.drop(columns=["id", "target"]).columns
target_col = "target"
X_train, X_valid, y_train, y_valid = dask_train_test_split(
    train_data[feature_cols], 
    train_data[target_col], 
    test_size=0.3
)

print(f'X_trainのshape:{X_train.shape}')
print(f'y_trainのshape:{y_train.shape}')
print(f'X_validのshape:{X_valid.shape}')
print(f'y_validのshape:{y_valid.shape}')

X_trainのshape:(Delayed('int-c13bb95c-68c3-4056-9006-1db25a7880df'), 695)
y_trainのshape:(dd.Scalar<size-ag..., dtype=int64>,)
X_validのshape:(Delayed('int-56d79577-dbfc-40bb-bd34-df1d701eb318'), 695)
y_validのshape:(dd.Scalar<size-ag..., dtype=int64>,)


/Users/ryusuke/anaconda3/envs/work/lib/python3.9/site-packages/dask_ml/model_selection/_split.py:462: FutureWarning: The default value for 'shuffle' must be specified when splitting DataFrames. In the future DataFrames will automatically be shuffled within blocks prior to splitting. Specify 'shuffle=True' to adopt the future behavior now, or 'shuffle=False' to retain the previous behavior.
  warnings.warn(


In [21]:
print(type(X_train))
print(type(y_train))

<class 'dask.dataframe.core.DataFrame'>
<class 'dask.dataframe.core.Series'>


In [22]:
# モデルの設定
model = lgb.DaskLGBMRegressor(n_estimators=10)

# daskの設定
cluster = LocalCluster(n_workers=2)
client = Client(cluster)

# モデルの学習
%time model.fit(X_train, y_train)

/Users/ryusuke/anaconda3/envs/work/lib/python3.9/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 52687 instead
  warnings.warn(


CPU times: user 6 µs, sys: 3 µs, total: 9 µs
Wall time: 36 µs


2023-08-21 23:38:34,588 - distributed.nanny.memory - WARNING - Worker tcp://127.0.0.1:52696 (pid=7985) exceeded 95% memory budget. Restarting...
2023-08-21 23:38:35,993 - distributed.nanny - WARNING - Restarting worker
2023-08-21 23:38:44,074 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 5.56 GiB -- Worker memory limit: 8.00 GiB
2023-08-21 23:38:44,173 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 5.64 GiB -- Worker memory limit: 8.00 GiB
2023-08-21 23:38:44,272 - distributed.worker.memory - WARNING - Unmanaged m

AttributeError: 'Future' object has no attribute 'get_params'

In [57]:
# 精度
y_train_pred = model.predict(X_train)
y_valid_pred = model.predict(X_valid)
print(f"trainの精度：{cosine_similarity(y_train, y_train_pred)}")
print(f"validの精度：{cosine_similarity(y_valid, y_valid_pred)}")

trainの精度：0.30204719310715356
validの精度：0.10335585223513896


## テストデータの予測

In [58]:
# 予測
id_test = test_data["id"].values.reshape(-1)
X_test = test_data.drop(columns=["id"]).copy()
X_test_selected = selector.transform(X_test)

In [59]:
y_test_pred = model.predict(X_test_selected)

In [60]:
# 保存
result = pd.DataFrame(data=y_test_pred, index=id_test, columns=["target"]).reset_index().rename(columns={"index": "id"})
result.to_csv(Path(RESULT_PATH, "20230820_LGBM_feature_selection.csv"), index=False)

# 参考サイト
- https://qiita.com/rockhopper/items/a68ceb3248f2b3a41c89